# Binning with MetaBat

Now we have all ingredients to continue binning: the scaffolds and bam files containing reads mapped on those scaffolds. 
In metagenomics, binning is the process of grouping reads or contigs and assigning them to operational taxonomic units. 
Binning methods can be based on either compositional features, alignment (similarity), or both.
`metabat2` uses both the contig depth and tetra-nucleotide frequencies to bin the contigs. 
Every bin will ideally represent one microbial genome from one particular microbe that was in the original DNA extraction.

**[DO:]** Run the script jgi_summarize_bam_contig_depths to calculate the average depth per contig.

In [ ]:
jgi_summarize_bam_contig_depths -h

calculate the contig depth per scaffold, use the `--outputDepth` option. 
Notice that the help page tells you to supply an arg(ument) where to store the depth output file. 
Specify a path to a file that this script will create. Something like this:

> ./script --outputDepth /path/to/depth_matrix

hint: a glob looks like this directory name/* and includes all files included in the directory

Try using the `*` with `ls` first. List all sorted bam files you created

In [ ]:
ls ./

In [ ]:
jgi_summarize_bam_contig_depths

The output of this process is the input for MetaBAT in the next step. 
After the jgi script finishes, make sure you check that the file contains a table. 
If so, please remove all BAM files. We don't need these anymore.

In [ ]:
head ..path/to/your/deth_matrix... # <-- substitute this path for the file you created with the jgi script.

In [ ]:
rm ./data/sorted # double- & triple-check that your depth_matrix is OK before you remove your sorted bam files

Now you have your depth_matrix, let's take a moment and reflect upon what this matrix does and how it helps in binning the microbial contigs. 
For this part, we will visualise the depth_matrix file in excel (or a similar spreadsheet editor) on your computer.

Follow these steps:
1. Download the depth_matrix to your personal computer.
2. The depth matrix is a big table in which columns are delimited by TABs. Open your data in excel and make sure all data is displayed as columns
3. A: What do the columns represent?
3. B: What do the rows represent?
4. For clarity, remove all columns except those that display the depth data.
5. Check if you have one column per sample.
6. Find the option for conditional formatting, filling the cells with colour depending on their content.
7. Color all cells in the excel sheet according to a colour gradient with three colors.
8. A: Can you identify two rows with a similar colour pattern,
8. B: what does that mean if these two have a similar colour pattern?

Now move on to binning part2!